In [33]:
import os             
import sys            
import threading      
import cv2            
import numpy as np    
import yaml           
import pickle
import time
import csv
import pdb
from joblib import Parallel, delayed

In [34]:
def get_surf_features_from_video(downsampled_video_filename, surf_feat_video_filename, keyframe_interval):                                                      
    "Receives filename of downsampled video and of output path for features. Extracts features in the given keyframe_interval. Saves features in pickled file." 
    keyframe_iterator = get_keyframes(downsampled_video_filename, keyframe_interval)
    
#     feature_vectors = np.

In [35]:
def get_keyframes(surf, downsampled_video_filename, keyframe_interval): 
    "Generator function which returns the next keyframe."         

    # Create video capture object                                     
    video_cap = cv2.VideoCapture(downsampled_video_filename)                                     
    frame = 0                                                                                    
    while True:                                                                                  
        frame += 1                                                                               
        ret, img = video_cap.read()                                                              
        if ret is False:                                                                         
            break                                                                                
        if frame % keyframe_interval == 0:                                                       
            surf_features = surf.detectAndCompute(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), None)[1]
            if surf_features is not None:                                                        
                yield surf_features.reshape((-1,128))                                            
#               yield img
    video_cap.release()


In [36]:
all_video_names = "tryvideos"
config_file = "config.yaml"                                                                  
my_params = yaml.load(open(config_file))                                                   
                                                                                           
# Get parameters from config file                                                          
keyframe_interval = my_params.get('keyframe_interval')                                     
hessian_threshold = my_params.get('hessian_threshold')                                     
surf_features_folderpath = my_params.get('surf_features')                                  
downsampled_videos = my_params.get('downsampled_videos')                                   


In [50]:
def get_for_one(line, surf):
    start = time.time()
    video_name = line.replace('\n', '')
    downsampled_video_filename = os.path.join(downsampled_videos, video_name + '.ds.mp4')
    surf_feat_video_filename = os.path.join(surf_features_folderpath, video_name + '.surf')

    if (os.path.isfile(surf_feat_video_filename)) or (not os.path.isfile(downsampled_video_filename)):
        print("skipping", video_name)
        return
#         continue

    # Get SURF features for one video
#         start = time.time()
#         keyframe_iterator = get_keyframes(surf, downsampled_video_filename, keyframe_interval)
#         print(time.time() - start)
#         start = time.time()
#         surf_feats = np.vstack([x for x in keyframe_iterator])
#         print(time.time() - start)
#         start = time.time()
#         np.savetxt("try.csv", surf_feats)
#         print(time.time() - start)
#         with open("try.csv", "w") as file1:
#             writes = csv.writer(file1, delimiter=',',)
#             writes.writerows(keyframe_iterator)

#         get_surf_features_from_video(downsampled_video_filename, surf_feat_video_filename, keyframe_interval)
    start = time.time()
    keyframe_iterator = get_keyframes(surf, downsampled_video_filename, keyframe_interval)
    print("iterator: ", time.time() - start)
    try:
        start = time.time()
        surf_feats = np.vstack(np.array(list(keyframe_iterator)))
        print("vstack: ", time.time() - start)
        start = time.time()
        np.savetxt(surf_feat_video_filename, surf_feats, delimiter=",")
        print("save file: ", time.time() - start)
        print("Finished", video_name, "in", time.time() - start, "seconds")
    except(ValueError):
        os.system("echo " + video_name + " > emptyvideos")

def get_features():                                                     
    # TODO: Create SURF object                                                                 
    # global surf
    # surf = cv2.SURF(12000,2,3,1)                                                        
    surf = cv2.xfeatures2d.SURF_create()
    surf.setExtended(True)

    # Check if folder for SURF features exists                                                 
    if not os.path.exists(surf_features_folderpath):                                           
        os.mkdir(surf_features_folderpath)                                                     

    # Loop over all videos (training, val, testing)                                            
    # TODO: get SURF features for all videos but only from keyframes                           

    fread = open(all_video_names, "r")
    Parallel(n_jobs=6, prefer="threads")(delayed(get_for_one)(line, surf) for line in fread.readlines())
#     for line in fread.readlines():

In [51]:
start = time.time()
get_features()
print(time.time() - start)

iterator:  1.430511474609375e-06
iterator:  1.430511474609375e-06
iterator:  1.430511474609375e-06
vstack:  4.533970355987549
vstack:  6.1893346309661865
save file:  7.705695390701294
Finished HVC1945 in 7.705829620361328 seconds
vstack:  12.836823463439941
save file:  10.007382869720459
Finished HVC1062 in 10.007795095443726 seconds
save file:  18.919201135635376
Finished HVC1012 in 18.919522762298584 seconds
31.846588373184204


In [67]:
# a = np.empty((0, 128), float)
# start = time.time()
# for i in keyframe_iterator:
#     a = np.vstack((a, i))
# print(a.shape)
# print(time.time() - start)
# #     (kps, descs) = surf.detectAndCompute(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY), None)
# #     descs.reshape((-1,128))
# #     np.hstack([a, i])
# # a = map(lambda x: surf.detectAndCompute(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY), None)[1].reshape((-1,128)), keyframe_iterator)
# # a = map(lambda x: surf.detectAndCompute(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY), None)[1].reshape((-1,128)), keyframe_iterator)
# # c = np.fromiter((surf.detectAndCompute(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY), None)[1].reshape((-1,128)) for i in keyframe_iterator), keyframe_iterator.dtype)
# # np.apply_along_axis(surf.detectAndCompute(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)))

In [68]:
# # np.fromiter(keyframe_iterator, dtype = np.dtype('f8',(128)))

# start = time.time()
# a = np.vstack(np.array(list(keyframe_iterator)))
# print(time.time() - start)

In [69]:
# a.shape

In [70]:
# start = time.time()
# np.savetxt("foo.csv", a, delimiter=",")
# print(time.time() - start)

In [71]:
# b = np.loadtxt("foo.csv", delimiter=",")